本ノートブックは以下を参考にして実行しました。

https://www.kaggle.com/code/vanguarde/h-m-eda-first-look/notebook

## 課題
トレーニングデータの期間の直後7日間でユーザーがなんの商品を購入するのか推定する

## ソリューション
レコメンデーション機能を使った、「このユーザーはこのような商品を買うだろう」という推定を行う。

## レコメンデーションのアプローチ方法(一般論)
- パーソナライズ(ユーザーごとにレコメンド)
- 非パーソナライズ(万人に向けたレコメンド)
- 協調型(他のユーザの嗜好行動をもとに、対象ユーザーへのアイテムをレコメンド)
- 内容ベース型(アイテムの特徴・説明とユーザの特徴・行動を用いてレコメンド)

参考：https://www.slideshare.net/takemikami/ss-76817490

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
pd.set_option('display.max_columns', 50)
pd.options.display.max_rows = 50
BASE_PATH = "../input/h-and-m-personalized-fashion-recommendations/"

In [ ]:
! ls /kaggle/input/h-and-m-personalized-fashion-recommendations


In [ ]:
articles_df = pd.read_csv(BASE_PATH+'articles.csv')
transactions_train_df = pd.read_csv(BASE_PATH+'transactions_train.csv')
customers_df = pd.read_csv(BASE_PATH+'customers.csv')
#sample_submission_df = pd.read_csv(BASE_PATH+'sample_submission.csv')

## articles.csv
1. article_id : 商品画像を識別するための番号
2. product_code : 商品を識別するためのコード
3. prod_name : 商品名
4. product_type_no : 商品種別番号
5. product_type_name : 商品種別名
6. product_group_name : 商品グループ名
7. graphical_appearance_no : 模様番号
8. graphical_appearance_name : 模様名
9. colour_group_code : カラーグループ番号
10. colour_group_name : カラー名
11. perceived_colour_value_id : 識別カラー値番号
12. perceived_colour_value_name : 識別カラー値名称
13. perceived_colour_master_id : 識別カラーマスター番号
14. perceived_colour_master_name : 識別カラーマスター名
15. department_no : 部門番号
16. department_name : 部門名
17. index_code : インデックスコード
18. index_name : インデックス名
19. index_group_no : インデックスグループ番号
20. index_group_name : インデックスグループ名
21. section_no : セクション番号
22. section_name : セクション名
23. garment_group_no : 衣類グループ番号
24. garment_group_name : 衣類グループ名
25. detail_desc : 詳細説明

In [ ]:
articles_df.head()

In [ ]:
articles_df.info()

商品カテゴリの種類と数

In [ ]:
f, ax = plt.subplots(figsize=(10,7))
ax = sns.histplot(data=articles_df,y='index_name', color='orange')
ax.set_xlabel('count by index name')
ax.set_ylabel('index name')
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(10,7))
ax = sns.histplot(data=articles_df,y='garment_group_name', color='orange',hue='index_group_name', multiple="stack")
ax.set_xlabel('count by garment group')
ax.set_ylabel('garment group')
plt.show()

In [ ]:
articles_df.groupby(['index_group_name', 'index_name']).count()['article_id']

In [ ]:
pd.options.display.max_rows = None
articles_df.groupby(['product_group_name', 'product_type_name']).count()['article_id']

In [ ]:
for col in articles_df.columns:
    if not 'no' in col and not 'code' in col and not 'id' in col:
        un_n = articles_df[col].nunique()
        print(f'n of unique {col}: {un_n}')

## transactions_train.csv
1. t_dat : 取引の日付
2. customer_id : 顧客番号
3. article_id : 商品画像を識別するための番号
4. price : 商品価格
5. sales_channel_id : 販売チャネル

In [ ]:
transactions_train_df.head()

In [ ]:
transactions_train_df.info()

トランザクションの価格について詳細を確認する。

In [ ]:
pd.set_option('display.float_format', '{:.4f}'.format)
transactions_train_df.describe()['price']

In [ ]:
sns.set_style("darkgrid")
f, ax = plt.subplots(figsize=(10,5))
ax = sns.boxplot(data=transactions_train_df, x='price', color='orange')
ax.set_xlabel('Price outliers')
plt.show()

見方
https://cdn-ak.f.st-hatena.com/images/fotolife/m/mizti/20171118/20171118190630.png


取引回数が多かったカスタマー上位10名

In [ ]:
transactions_byid = transactions_train_df.groupby('customer_id').count()
transactions_byid.sort_values(by='price', ascending=False)['price'][:10]

売り上げ高が多かったカスタマー上位10名

In [ ]:
transactions_byid_sum = transactions_train_df.groupby('customer_id').sum()
transactions_byid_sum.sort_values(by='price', ascending=False)['price'][:10]

商品ごとに価格帯が大きく異なるため、商品アセットごとに上記の値をみた方がより良い
articles.csvに商品のグループ情報があるのでマージする

In [ ]:
articles_for_merge = articles_df[['article_id', 'prod_name', 'product_type_name', 'product_group_name', 'index_name']]

In [ ]:
articles_for_merge = transactions_train_df[['customer_id', 'article_id', 'price', 't_dat']].merge(articles_for_merge, on='article_id', how='left')

これで商品グループごとに購入される価格帯を比較できる

In [ ]:
sns.set_style("darkgrid")
f, ax = plt.subplots(figsize=(25,18))
ax = sns.boxplot(data=articles_for_merge, x='price', y='product_group_name')
ax.set_xlabel('Price outliers', fontsize=22)
ax.set_ylabel('Index names', fontsize=22)
ax.xaxis.set_tick_params(labelsize=22)
ax.yaxis.set_tick_params(labelsize=22)

plt.show()

アクセサリーに関してはさまざまな小物が存在することが想定されるため、さらに詳細に商品タイプを調べる

In [ ]:
sns.set_style("darkgrid")
f, ax = plt.subplots(figsize=(25,18))
_ = articles_for_merge[articles_for_merge['product_group_name'] == 'Accessories']
ax = sns.boxplot(data=_, x='price', y='product_type_name')
ax.set_xlabel('Price outliers', fontsize=22)
ax.set_ylabel('Index names', fontsize=22)
ax.xaxis.set_tick_params(labelsize=22)
ax.yaxis.set_tick_params(labelsize=22)
del _

plt.show()

平均価格が最も高い指標は「婦人服」。最安値は子供服。

In [ ]:
articles_index = articles_for_merge[['index_name', 'price']].groupby('index_name').mean()
sns.set_style("darkgrid")
f, ax = plt.subplots(figsize=(10,5))
ax = sns.barplot(x=articles_index.price, y=articles_index.index, color='orange', alpha=0.8)
ax.set_xlabel('Price by index')
ax.set_ylabel('Index')
plt.show()

平均価格が最も低いのは「文房具」、最も高いのは「靴」。

In [ ]:
articles_index = articles_for_merge[['product_group_name', 'price']].groupby('product_group_name').mean()
sns.set_style("darkgrid")
f, ax = plt.subplots(figsize=(10,5))
ax = sns.barplot(x=articles_index.price, y=articles_index.index, color='orange', alpha=0.8)
ax.set_xlabel('Price by product group')
ax.set_ylabel('Product group')
plt.show()

次に、平均価格の上位5つの製品グループの時間的な平均価格の変化を確認する
- Shoes
- Garment Full body
- Bags
- Garment Lower body
- Underwear/nightwear

In [ ]:
articles_for_merge['t_dat'] = pd.to_datetime(articles_for_merge['t_dat'])

In [ ]:
product_list = ['Shoes', 'Garment Full body', 'Bags', 'Garment Lower body', 'Underwear/nightwear']
colors = ['cadetblue', 'orange', 'mediumspringgreen', 'tomato', 'lightseagreen']
k = 0
f, ax = plt.subplots(3, 2, figsize=(20, 15))
for i in range(3):
    for j in range(2):
        try:
            product = product_list[k]
            articles_for_merge_product = articles_for_merge[articles_for_merge.product_group_name == product_list[k]]
            series_mean = articles_for_merge_product[['t_dat', 'price']].groupby(pd.Grouper(key="t_dat", freq='M')).mean().fillna(0)
            series_std = articles_for_merge_product[['t_dat', 'price']].groupby(pd.Grouper(key="t_dat", freq='M')).std().fillna(0)
            ax[i, j].plot(series_mean, linewidth=4, color=colors[k])
            ax[i, j].fill_between(series_mean.index, (series_mean.values-2*series_std.values).ravel(), 
                             (series_mean.values+2*series_std.values).ravel(), color=colors[k], alpha=.1)
            ax[i, j].set_title(f'Mean {product_list[k]} price in time')
            ax[i, j].set_xlabel('month')
            ax[i, j].set_xlabel(f'{product_list[k]}')
            k += 1
        except IndexError:
            ax[i, j].set_visible(False)
plt.show()

見方：https://www.pref.niigata.lg.jp/uploaded/attachment/228339.pdf


## customers.csv
1. customer_id : 顧客番号
2. FN : ?
3. Active : ?
4. club_member_status : メンバー会員のステータス
5. fashion_news_frequency : ?
6. age : 年齢
7. postal_code : 郵便番号(暗号化されている)


In [ ]:
customers_df.head()

重複したカスタマーIDを持っている人はいない

In [ ]:
customers_df.shape[0]- customers_df['customer_id'].nunique()

一人だけ異常な住所番号や、カスタマーIDを持つ人がいるがこれは、センターなどのシステムアカウントかもしれない。

In [ ]:
data_postal = customers_df.groupby('postal_code', as_index=False).count().sort_values('customer_id', ascending=False)
data_postal.head()

一番利用している年代は21-23歳

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
sns.set_style("darkgrid")
f, ax = plt.subplots(figsize=(10,5))
ax = sns.histplot(data=customers_df, x='age', bins=50, color='orange')
ax.set_xlabel('Distribution of the customers age')
plt.show()

ほとんどの人のクラブ会員が有効の状態


In [ ]:
sns.set_style("darkgrid")
f, ax = plt.subplots(figsize=(10,5))
ax = sns.histplot(data=customers_df, x='club_member_status', color='orange')
ax.set_xlabel('Distribution of club member status')
plt.show()

`customers_df['fashion_news_frequency']`の`nan`に表記揺れが発生しているので、統一しておく

In [ ]:
customers_df['fashion_news_frequency'].unique()

In [ ]:
#反転処理
customers_df.loc[~customers_df['fashion_news_frequency'].isin(['Regularly', 'Monthly']), 'fashion_news_frequency'] = 'None'
customers_df['fashion_news_frequency'].unique()

キャンペーンのメッセージを受け取らないとしている人が多くいる

In [ ]:
pie_data = customers_df[['customer_id', 'fashion_news_frequency']].groupby('fashion_news_frequency').count()

In [ ]:
sns.set_style("darkgrid")
f, ax = plt.subplots(figsize=(10,5))
# ax = sns.histplot(data=customers, x='fashion_news_frequency', color='orange')
# ax = sns.pie(data=customers, x='fashion_news_frequency', color='orange')
colors = sns.color_palette('pastel')
ax.pie(pie_data.customer_id, labels=pie_data.index, colors = colors)
ax.set_facecolor('lightgrey')
ax.set_xlabel('Distribution of fashion news frequency')
plt.show()

最後に購入した商品を最大価格と最小価格で確認する

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
max_price_ids = transactions_train_df[transactions_train_df.t_dat==transactions_train_df.t_dat.max()].sort_values('price', ascending=False).iloc[:5][['article_id', 'price']]
min_price_ids = transactions_train_df[transactions_train_df.t_dat==transactions_train_df.t_dat.min()].sort_values('price', ascending=True).iloc[:5][['article_id', 'price']]

最終日に購入された上位５つの商品を確認

In [ ]:
f, ax = plt.subplots(1, 5, figsize=(20,10))
i = 0
for _, data in max_price_ids.iterrows():
    desc = articles_df[articles_df['article_id'] == data['article_id']]['detail_desc'].iloc[0]
    desc_list = desc.split(' ')
    for j, elem in enumerate(desc_list):
        if j > 0 and j % 5 == 0:
            desc_list[j] = desc_list[j] + '\n'
    desc = ' '.join(desc_list)
    img = mpimg.imread(f'../input/h-and-m-personalized-fashion-recommendations/images/0{str(data.article_id)[:2]}/0{int(data.article_id)}.jpg')
    ax[i].imshow(img)
    ax[i].set_title(f'price: {data.price:.2f}')
    ax[i].set_xticks([], [])
    ax[i].set_yticks([], [])
    ax[i].grid(False)
    ax[i].set_xlabel(desc, fontsize=10)
    i += 1
plt.show()

最も過去に購入された上位５つの商品を確認

In [ ]:
f, ax = plt.subplots(1, 5, figsize=(20,10))
i = 0
for _, data in min_price_ids.iterrows():
    desc = articles_df[articles_df['article_id'] == data['article_id']]['detail_desc'].iloc[0]
    desc_list = desc.split(' ')
    for j, elem in enumerate(desc_list):
        if j > 0 and j % 4 == 0:
            desc_list[j] = desc_list[j] + '\n'
    desc = ' '.join(desc_list)
    img = mpimg.imread(f'../input/h-and-m-personalized-fashion-recommendations/images/0{str(data.article_id)[:2]}/0{int(data.article_id)}.jpg')
    ax[i].imshow(img)
    ax[i].set_title(f'price: {data.price:.4f}')
    ax[i].set_xlabel(desc, fontsize=10)
    ax[i].set_xticks([], [])
    ax[i].set_yticks([], [])
    ax[i].grid(False)
    i += 1
plt.axis('off')
plt.show()

読んでいただきありがとうございました。